In [1]:
!pip install elasticsearch

     |████████████████████████████████| 385 kB 12.1 MB/s 


In [1]:
import pandas as pd

In [2]:
import json

In [3]:
import numpy as np

In [4]:
from elasticsearch import Elasticsearch , helpers

In [5]:
import time

In [6]:
import elasticsearch

In [7]:
elasticsearch.__version__

(7, 16, 1)

In [8]:
import pprint  
INDEX_NAME = "mbn_index2"


INDEX_SETTINGS = {
  "settings" : {
    "index":{
      "analysis":{
        "analyzer":{
          "korean":{
            "type":"custom",
            "tokenizer":"nori_tokenizer",
            "filter": [ "shingle" ],

          }
        }
      }
    }
  },
  "mappings": {

      "properties" : {
        "CONTEXT" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        },
        "TITLE" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        },
        "DATE" :{
            "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        },
        "CATEGORY" :{
            "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        }
      }

  }
}

In [10]:
DOCS = json.load(open("../app/data/mbn_news_wiki.json" , 'r'))

In [11]:
try:
    es.transport.close()
except:
    pass
es = Elasticsearch()

In [12]:
es.info()

/opt/conda/envs/lightweight/lib/python3.7/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': '3af703a614fe',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'PWprZm19T4aX3mtMwvpgiw',
 'version': {'number': '7.16.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '5b38441b16b1ebb16a27c107a4c3865776e20c53',
  'build_date': '2021-12-11T00:29:38.865893768Z',
  'build_snapshot': False,
  'lucene_version': '8.10.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [13]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

/opt/conda/envs/lightweight/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  """Entry point for launching an IPython kernel.
/opt/conda/envs/lightweight/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  This is separate from the ipykernel package so we can avoid doing imports until


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'mbn_index2'}

In [14]:
def is_nan(key,values) :
    if values['CONTEXT']!=values['CONTEXT'] or values['CATEGORY']!=values['CATEGORY']  or values['DATE']!=values['DATE']  or values['TITLE']!=values['TITLE'] :
    # if np.isnan(values['CONTEXT']) :
        values['CONTEXT'] = '0'
        values['DATE'] = '0'
        values['TITLE'] = '0'
        print(id)



In [15]:
for id, docs in DOCS.items():
    is_nan(id,docs)

641143
726024


In [16]:
news_wiki =[
    {
        "_index" : INDEX_NAME,
        "_id" : doc_id ,
        "_source" :{
            "context" : doc['CONTEXT'],
            "title" : doc['TITLE'] ,
            "date" : doc['DATE'] ,
            "category" : doc['CATEGORY']
        }
    } for doc_id , doc in DOCS.items()
]

In [17]:
start = time.time() 
try :
    
    response = helpers.bulk(es,news_wiki)
    print("\nRESPONSE" , response)
    print("time :", time.time() - start)
except Exception as e :
    print("\nError:  " , e )
    print("time :", time.time() - start)


RESPONSE (842904, [])
time : 192.9208116531372


In [18]:
INDEX_NAME

'mbn_index2'

In [19]:
query = "사랑하지만 힘들어 죽겠네"
res = es.search(index=INDEX_NAME, q=query, size=10)
pprint.pprint(res)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '205730',
                    '_ignored': ['context.keyword'],
                    '_index': 'mbn_index2',
                    '_score': 24.373047,
                    '_source': {'category': '스포츠',
                                'context': '[매경닷컴 MK스포츠(日 오키나와)=] 26일 일본 킨 '
                                           '구장에서 SK 와이번스와 KIA 타이거즈의 연습경기가 '
                                           '열렸다.4회초 KIA 백용환이 3루에 진루하고 숨을 고르고 '
                                           '있다.SK 와이번스는 오키나와 캠프에서 일본 프로팀과 4번, '
                                           '국내 프로팀과 6번 총 10번의 연습경기를 치르면서 실전감각을 '
                                           '점검한다.한편, KIA 타이거즈 선수단은 일본 오키나와에서 '
                                           '국내외 팀들과 총 12차례 연습경기를 가진다. 오는 3일 '
                                           "인천국제공항을 통해 귀국할 예정이다][ ]백용환 '힘들어 "
                                           "죽겠네' [MK포토]",
                  

In [9]:
# import time
# count = 0
# for doc_id, doc in DOCS.items():
#     es.index(index=INDEX_NAME,  id=doc_id, body=doc)
#     time.sleep(0.001)
#     print(count)
#     count+=1


598417
598418
598419
598420
598421
598422
598423
598424
598425
598426
598427
598428
598429
598430
598431
598432
598433
598434
598435
598436
598437
598438
598439
598440
598441
598442
598443
598444
598445
598446
598447
598448
598449
598450
598451
598452
598453
598454
598455
598456
598457
598458
598459
598460
598461
598462
598463
598464
598465
598466
598467
598468
598469
598470
598471
598472
598473
598474
598475
598476
598477
598478
598479
598480
598481
598482
598483
598484
598485
598486
598487
598488
598489
598490
598491
598492
598493
598494
598495
598496
598497
598498
598499
598500
598501
598502
598503
598504
598505
598506
598507
598508
598509
598510
598511
598512
598513
598514
598515
598516
598517
598518
598519
598520
598521
598522
598523
598524
598525
598526
598527
598528
598529
598530
598531
598532
598533
598534
598535
598536
598537
598538
598539
598540
598541
598542
598543
598544
598545
598546
598547
598548
598549
598550
598551
598552
598553
598554
598555
598556
598557
598558
59855

RequestError: RequestError(400, 'mapper_parsing_exception', 'failed to parse')

In [9]:
if es.indices.exists(INDEX_NAME):
    print(True)

True
